# Aimes Iowa Housing Price Prediction

## Introduction:

This notebook analysis Aimes House sale prices. The data is for residential properties sale prices from 2006 to 2010 in Aimes, Iowa. The data contains more than 80 features that evaluates houses sale prices for 2051 houses.  

### Problem Statement:

The value of a house is often estimated by several personals who come to check the size, quality, condition, features, location of the house. While this method doesn't give accurate scores, it also makes it hard for home buyers and sellers to know the true value of the property.

One of the websites that lead buying/selling houses in the U.S. is Zillow. It can estimate the prices for houses that are on and off market with very high accuracy. Currently, Zestimate, our house predication platform, can estimate house prices for on-market houses with %1.5 maiden error, and %7.5 for off-market houses. The better lower these error scores are, the better our house price predictions are, which yields into more people using our platform and more money for us.

Lets begin, shall we?

## Importing Libraries  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_rows = 100 #shows 100 rows max

## Reading Files

In [2]:
#reading the csv
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

In [3]:
#lets see what we have
train.shape

(2051, 81)

In [4]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [5]:
train['MS Zoning'].value_counts()

RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: MS Zoning, dtype: int64

In [6]:
#checks if column names are the same and also checks for any extra columns.
for i in train.columns:
    if i not in test.columns: print(i)
    

SalePrice


From the shape of test and train data, and also the above for loop, we can see that all columns match between them and SalePrice only exist in the train data

I will combine both of the dataFrames to clean them at the same time, then i will split them.

In [7]:
# storing the number of columns in each dataFrame to split them later
train_rows = train.shape[0] 
test_rows = test.shape[0]

#concating both dataFrames and reseting the index
data = pd.concat((train, test), sort=False).reset_index(drop=True)

#dropping the SalePrice column. i will clean the SalePrice on the train df after splitting it
data.drop(['SalePrice'], axis=1, inplace=True)

In [8]:
#they are concatenated
data.shape

(2930, 80)

In [9]:
data.tail()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
2925,1662,527377110,60,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,11,2007,WD
2926,1234,535126140,60,RL,90.0,14670,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,MnPrv,NaN,0,8,2008,WD
2927,1373,904100040,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2008,WD
2928,1672,527425140,20,RL,60.0,9000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,GdWo,NaN,0,5,2007,WD
2929,1939,535327160,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,GdWo,NaN,0,3,2007,WD


In [10]:
#checking the type of each column
data.dtypes

Id                   int64
PID                  int64
MS SubClass          int64
MS Zoning           object
Lot Frontage       float64
Lot Area             int64
Street              object
Alley               object
Lot Shape           object
Land Contour        object
Utilities           object
Lot Config          object
Land Slope          object
Neighborhood        object
Condition 1         object
Condition 2         object
Bldg Type           object
House Style         object
Overall Qual         int64
Overall Cond         int64
Year Built           int64
Year Remod/Add       int64
Roof Style          object
Roof Matl           object
Exterior 1st        object
Exterior 2nd        object
Mas Vnr Type        object
Mas Vnr Area       float64
Exter Qual          object
Exter Cond          object
Foundation          object
Bsmt Qual           object
Bsmt Cond           object
Bsmt Exposure       object
BsmtFin Type 1      object
BsmtFin SF 1       float64
BsmtFin Type 2      object
B

## EDA

In [11]:
#looking for the number of missing data more closely
data.isnull().sum().sort_values(ascending = False).head(20)

Pool QC           2917
Misc Feature      2824
Alley             2732
Fence             2358
Fireplace Qu      1422
Lot Frontage       490
Garage Yr Blt      159
Garage Qual        159
Garage Cond        159
Garage Finish      159
Garage Type        157
Bsmt Exposure       83
BsmtFin Type 2      81
BsmtFin Type 1      80
Bsmt Cond           80
Bsmt Qual           80
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Full Bath       2
Bsmt Half Bath       2
dtype: int64

In [12]:
#looking for the percentage of missing data
data.isnull().mean().sort_values(ascending= False)


Pool QC            0.995563
Misc Feature       0.963823
Alley              0.932423
Fence              0.804778
Fireplace Qu       0.485324
Lot Frontage       0.167235
Garage Yr Blt      0.054266
Garage Qual        0.054266
Garage Cond        0.054266
Garage Finish      0.054266
Garage Type        0.053584
Bsmt Exposure      0.028328
BsmtFin Type 2     0.027645
BsmtFin Type 1     0.027304
Bsmt Cond          0.027304
Bsmt Qual          0.027304
Mas Vnr Type       0.007850
Mas Vnr Area       0.007850
Bsmt Full Bath     0.000683
Bsmt Half Bath     0.000683
Bsmt Unf SF        0.000341
BsmtFin SF 2       0.000341
Electrical         0.000341
BsmtFin SF 1       0.000341
Total Bsmt SF      0.000341
Garage Area        0.000341
Garage Cars        0.000341
Exter Qual         0.000000
Condition 1        0.000000
PID                0.000000
MS SubClass        0.000000
MS Zoning          0.000000
Lot Area           0.000000
Street             0.000000
Lot Shape          0.000000
Land Contour       0

There are many columns with very high missing data percentage

To clean them up, 
- We will be replacing qualitative data types with None. where None indicates that that specific house doesn't have this feature. Datatypes such as: Pool QC, Alley, Misc Feature, Fence...

- We will also replace quantitative data types with 0, indicating that there is nothing for this feature. such as: Garage Area, Cars, year built ...

#### 1. Qualitative:

- 'None' indicates that that specific house doesn't have this feature.


Checking if any of these columns have a specified name that indicates missing data:


In [13]:
data['Pool QC'].value_counts()

Gd    4
Ex    4
TA    3
Fa    2
Name: Pool QC, dtype: int64

In [14]:
data['Misc Feature'].value_counts()

Shed    95
Gar2     5
Othr     4
Elev     1
TenC     1
Name: Misc Feature, dtype: int64

In [15]:
data['Alley'].value_counts()

Grvl    120
Pave     78
Name: Alley, dtype: int64

In [16]:
data['Fence'].value_counts()

MnPrv    330
GdPrv    118
GdWo     112
MnWw      12
Name: Fence, dtype: int64

In [17]:
data['Fireplace Qu'].value_counts()

Gd    744
TA    600
Fa     75
Po     46
Ex     43
Name: Fireplace Qu, dtype: int64

In [18]:
data['Garage Finish'].value_counts()

Unf    1231
RFn     812
Fin     728
Name: Garage Finish, dtype: int64

In [19]:
train['Garage Cond'].value_counts()

TA    1868
Fa      47
Gd      12
Po       8
Ex       2
Name: Garage Cond, dtype: int64

In [20]:
data['Garage Qual'].value_counts()

TA    2615
Fa     124
Gd      24
Po       5
Ex       3
Name: Garage Qual, dtype: int64

In [21]:
data['Garage Type'].value_counts()

Attchd     1731
Detchd      782
BuiltIn     186
Basment      36
2Types       23
CarPort      15
Name: Garage Type, dtype: int64

In [22]:
#it has 'No'. it actaully means no exposure rather than no basement.
data['Bsmt Exposure'].value_counts()

No    1906
Av     418
Gd     284
Mn     239
Name: Bsmt Exposure, dtype: int64

In [23]:
data['BsmtFin Type 2'].value_counts()

Unf    2499
Rec     106
LwQ      89
BLQ      68
ALQ      53
GLQ      34
Name: BsmtFin Type 2, dtype: int64

In [24]:
data['BsmtFin Type 1'].value_counts()

GLQ    859
Unf    851
ALQ    429
Rec    288
BLQ    269
LwQ    154
Name: BsmtFin Type 1, dtype: int64

In [25]:
data['Bsmt Cond'].value_counts()

TA    2616
Gd     122
Fa     104
Po       5
Ex       3
Name: Bsmt Cond, dtype: int64

In [26]:
data['Bsmt Qual'].value_counts()

TA    1283
Gd    1219
Ex     258
Fa      88
Po       2
Name: Bsmt Qual, dtype: int64

In [27]:
# it has a None already
data['Mas Vnr Type'].value_counts()

None       1752
BrkFace     880
Stone       249
BrkCmn       25
CBlock        1
Name: Mas Vnr Type, dtype: int64

In [28]:
#Loops over each of the specific qualitative-data-type columns and
# replaces their missing values with None.
for column in ('Pool QC', 'Misc Feature', 'Alley', 
               'Fence', 'Fireplace Qu', 'Garage Finish',
               'Garage Cond','Garage Qual','Garage Type',
               'Bsmt Exposure','BsmtFin Type 2','BsmtFin Type 1',
               'Bsmt Cond','Bsmt Qual','Mas Vnr Type',
              ):
    data[column].fillna('None', inplace =True)


In [29]:
#looking for the percentage of missing data now 
data.isnull().mean().sort_values(ascending= False).head(20)

Lot Frontage      0.167235
Garage Yr Blt     0.054266
Mas Vnr Area      0.007850
Bsmt Half Bath    0.000683
Bsmt Full Bath    0.000683
Electrical        0.000341
BsmtFin SF 1      0.000341
Garage Cars       0.000341
Garage Area       0.000341
Total Bsmt SF     0.000341
Bsmt Unf SF       0.000341
BsmtFin SF 2      0.000341
Exter Cond        0.000000
Exter Qual        0.000000
Year Remod/Add    0.000000
Mas Vnr Type      0.000000
Foundation        0.000000
Bsmt Qual         0.000000
Bsmt Cond         0.000000
Bsmt Exposure     0.000000
dtype: float64

Better!!

#### 2. Quantitative:

In [30]:
#replaced the specified quantitative-data-types with 0 for missing value.
for column in ('Garage Yr Blt', 'Garage Area', 'Garage Cars', 
     'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Full Bath', 
     'Bsmt Half Bath', 'Bsmt Unf SF','Total Bsmt SF',
     "Mas Vnr Area" ):
     data[column].fillna(0, inplace =True)



In [31]:
# becuase every house should have a street connected to the property,
# missing data here means that there was proabably a proplem while entering
# the data.
data['Lot Frontage'].describe()

count    2440.000000
mean       69.224590
std        23.365335
min        21.000000
25%        58.000000
50%        68.000000
75%        80.000000
max       313.000000
Name: Lot Frontage, dtype: float64

In [32]:
#for this column, i will fill the nan values with the average of the column.
data['Lot Frontage'].fillna(train['Lot Frontage'].mean(), inplace = True)


In [33]:
data.isnull().mean().sort_values(ascending= False).head()

Electrical        0.000341
Sale Type         0.000000
Exter Qual        0.000000
Year Remod/Add    0.000000
Roof Style        0.000000
dtype: float64

In [34]:
#checking for unique values in the 'Electrical' column.
data['Electrical'].value_counts()

SBrkr    2682
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [35]:
#checking for missing values
data['Electrical'].isnull().sum()

1

In [36]:
#becasue there is only one null row, i am going to fill it with the most common value 'SBrkr'
data[data['Electrical'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
2686,1578,916386080,80,RL,73.0,9735,Pave,None,Reg,Lvl,...,0,0,0,None,None,None,0,5,2008,WD


In [37]:
#filling it
data['Electrical'].fillna('SBrkr', inplace=True)

In [38]:
data['Electrical'].isnull().sum()

0

In [39]:
data.isnull().sum().sum()

0

No more missing values!

### Saving the cleaned file

In [296]:
data.to_csv('../datasets/test_train_cleaned.csv', index=False)